In [4]:
# Returns list of BFS layers from given (full, unabridged) adjacency list <adj>

def bfs (adj, start=None, append_empty=True):
    # Precondition:
    assert adj
    
    # Track remaining unprocessed nodes
    remaining = {x for x in adj.keys ()}
    
    # Decide BFS starting root node
    if start in remaining:
        remaining.remove (start)
    else:
        start = remaining.pop ()
    
    # BFS layers and record of seen nodes
    layers = [{start}]
    seen = {start}
    
    while remaining:
        # Prepare to construct next_layer
        prev_layer = layers[-1]
        next_layer = set ()
        
        # Populate next_layer
        for node in prev_layer:
            next_layer |= adj[node]
        next_layer -= seen
        
        # Record next_layer
        remaining -= next_layer
        seen |= next_layer
        # Note: Append even if next_layer is empty.
        # An empty set will separate layers of
        # different connected components
        layers.append (next_layer)
        
        # Start new connected component
        if not next_layer and remaining:
            if not append_empty:
                layers.pop ()
            start = remaining.pop ()
            seen.add (start)
            layers.append ({start})
        
        # Remove final trailing empty set (applicable iff append_empty option set)
        elif not next_layer:
            layers.pop ()
            
    return layers

In [5]:
# Recursively forms nested BFS layers from each BFS layer

def recursive_bfs (adj, **kwargs):
    # Precondition:
    assert adj
    
    # Run BFS initially to get layers
    layers = bfs (adj, **kwargs)

    # Recursively decompose each BFS layer
    for i, layer in enumerate (layers):
        # Prepare layer-local adj
        # (i.e. restrict edges to nodes within this layer)
        local_adj = {node: adj[node] & layer for node in layer}
        
        # Done if there is one or fewer nodes
        if len (local_adj) <= 1:
            continue
        
        # Update layer with recursively decomposed result
        else:
            layers[i] = recursive_bfs (local_adj, **kwargs)

    return layers

In [22]:
# Convert RBFS layers into HTML representation

def render (rbfs_tree, input_base, depth=0):
    text = []
    
    # Record seen nodes just to double-check
    seen = set ()

    # Set div orientation and indent level
    indent = '  ' * (depth + 2)
    if depth % 2:
        orientation = 'horizontal'
    else:
        orientation = 'vertical'

    # Render each layer (recursively if needed)
    for i, layer in enumerate (rbfs_tree):
        
        # Connected component divider
        if not layer:
            text.append ('{}<div class="break {}"></div>'.format (indent, orientation))
        
        # Node
        elif isinstance (layer, set):
            index, = [x for x in layer]
            text.append ('{}<div id="node_{}" class="node {}">{}</div>'.format (indent, index, orientation, index))
            seen.update (layer)
            
        # Layer (must recurse deeper)
        else:
            rendered, _ = render (layer, input_base, depth=depth+1)
            text.append ('{}<div class="layer {}">\n{}\n{}</div>'.format (indent, orientation, rendered, indent))
            seen.update (_)
    
    # Join rendered layers
    text = '\n'.join (text)
    
    # For outermost level, wrap in container, apply stylesheet
    if depth == 0:
        template = '''\
<!DOCTYPE html>
<html>
<head>
  <title>BFS Decomposition: {}</title>
</head>
<body>
  <div class="container horizontal">
  <canvas></canvas>
{}
  </div>
  <link rel="stylesheet" type="text/css" href="../css/style.css">'
  <script type="module" src="../js/render.js"></script>
</body>
</html>'''
        text = template.format (input_base, text)
    
    return text, seen

In [23]:
# Generates html, json files for graph given in <input_file>

def process (input_file):
    
    from os.path import basename
    input_base = basename (input_file).split ('.')[0]
    
    #####################################################
    
    with open (input_file, 'r') as f:
        raw_lines = f.readlines ()
    
    #####################################################
    
    clauses = []

    for line in raw_lines:
        try:
            clause = [int (x) for x in line.split ()][:3]
            if len (clause) == 3:
                clauses.append (clause)
        except:
            pass
        
    #####################################################
    
    flattened = [x for clause in clauses for x in clause]
    
    #####################################################
    
    uniques = set (flattened)
    
    #####################################################
    
    from collections import defaultdict

    value2indices = defaultdict (list)

    for i,x in enumerate (flattened):
        value2indices[x].append (i)

    #####################################################
    
    from itertools import combinations

    adjacencies = defaultdict (set)

    for i, clause in enumerate (clauses):
        base = 3 * i
        for j, literal  in enumerate (clause):
            idx = base + j

            # A literal is adjacent to literals that are its complement...
            complement = value2indices[-literal]
            adjacencies[idx].update (complement)

            # ...and to the Literals in the same clause
            adjacencies[idx].update ({base + offset for offset in range (3)})
            adjacencies[idx].remove (idx)
    
    #####################################################
    
    abridged_adj = {k: {i for i in v if i > k} for k,v in adjacencies.items ()}
    
    #####################################################
    
    rbfs_layers = recursive_bfs (adjacencies)
    
    #####################################################
    
    html_file = 'html/{}.html'.format (input_base)
    
    #####################################################
    
    text, seen = render (rbfs_layers, input_base)

    with open (html_file, 'wt') as f:
        f.write (text)
        
    #####################################################
    
    edges_file = 'json/{}.json'.format (input_base)
    
    #####################################################
    
    import json
    with open (edges_file, 'wt') as f:
        json.dump ({node: sorted (adjacent) for node, adjacent in abridged_adj.items ()}, f, indent=2, sort_keys=True)

In [25]:
# Process all files in <data_dir>

from os import listdir, path

data_dir = 'uf20-91'

for input_file in listdir (data_dir):
    filepath = path.join (data_dir, input_file)
    print ('Processing {}'.format (filepath))
    process (filepath)

Processing uf20-91\uf20-01.cnf
Processing uf20-91\uf20-010.cnf
Processing uf20-91\uf20-0100.cnf
Processing uf20-91\uf20-01000.cnf
Processing uf20-91\uf20-0101.cnf
Processing uf20-91\uf20-0102.cnf
Processing uf20-91\uf20-0103.cnf
Processing uf20-91\uf20-0104.cnf
Processing uf20-91\uf20-0105.cnf
Processing uf20-91\uf20-0106.cnf
Processing uf20-91\uf20-0107.cnf
Processing uf20-91\uf20-0108.cnf
Processing uf20-91\uf20-0109.cnf
Processing uf20-91\uf20-011.cnf
Processing uf20-91\uf20-0110.cnf
Processing uf20-91\uf20-0111.cnf
Processing uf20-91\uf20-0112.cnf
Processing uf20-91\uf20-0113.cnf
Processing uf20-91\uf20-0114.cnf
Processing uf20-91\uf20-0115.cnf
Processing uf20-91\uf20-0116.cnf
Processing uf20-91\uf20-0117.cnf
Processing uf20-91\uf20-0118.cnf
Processing uf20-91\uf20-0119.cnf
Processing uf20-91\uf20-012.cnf
Processing uf20-91\uf20-0120.cnf
Processing uf20-91\uf20-0121.cnf
Processing uf20-91\uf20-0122.cnf
Processing uf20-91\uf20-0123.cnf
Processing uf20-91\uf20-0124.cnf
Processing uf2

Processing uf20-91\uf20-0327.cnf
Processing uf20-91\uf20-0328.cnf
Processing uf20-91\uf20-0329.cnf
Processing uf20-91\uf20-033.cnf
Processing uf20-91\uf20-0330.cnf
Processing uf20-91\uf20-0331.cnf
Processing uf20-91\uf20-0332.cnf
Processing uf20-91\uf20-0333.cnf
Processing uf20-91\uf20-0334.cnf
Processing uf20-91\uf20-0335.cnf
Processing uf20-91\uf20-0336.cnf
Processing uf20-91\uf20-0337.cnf
Processing uf20-91\uf20-0338.cnf
Processing uf20-91\uf20-0339.cnf
Processing uf20-91\uf20-034.cnf
Processing uf20-91\uf20-0340.cnf
Processing uf20-91\uf20-0341.cnf
Processing uf20-91\uf20-0342.cnf
Processing uf20-91\uf20-0343.cnf
Processing uf20-91\uf20-0344.cnf
Processing uf20-91\uf20-0345.cnf
Processing uf20-91\uf20-0346.cnf
Processing uf20-91\uf20-0347.cnf
Processing uf20-91\uf20-0348.cnf
Processing uf20-91\uf20-0349.cnf
Processing uf20-91\uf20-035.cnf
Processing uf20-91\uf20-0350.cnf
Processing uf20-91\uf20-0351.cnf
Processing uf20-91\uf20-0352.cnf
Processing uf20-91\uf20-0353.cnf
Processing uf

Processing uf20-91\uf20-0553.cnf
Processing uf20-91\uf20-0554.cnf
Processing uf20-91\uf20-0555.cnf
Processing uf20-91\uf20-0556.cnf
Processing uf20-91\uf20-0557.cnf
Processing uf20-91\uf20-0558.cnf
Processing uf20-91\uf20-0559.cnf
Processing uf20-91\uf20-056.cnf
Processing uf20-91\uf20-0560.cnf
Processing uf20-91\uf20-0561.cnf
Processing uf20-91\uf20-0562.cnf
Processing uf20-91\uf20-0563.cnf
Processing uf20-91\uf20-0564.cnf
Processing uf20-91\uf20-0565.cnf
Processing uf20-91\uf20-0566.cnf
Processing uf20-91\uf20-0567.cnf
Processing uf20-91\uf20-0568.cnf
Processing uf20-91\uf20-0569.cnf
Processing uf20-91\uf20-057.cnf
Processing uf20-91\uf20-0570.cnf
Processing uf20-91\uf20-0571.cnf
Processing uf20-91\uf20-0572.cnf
Processing uf20-91\uf20-0573.cnf
Processing uf20-91\uf20-0574.cnf
Processing uf20-91\uf20-0575.cnf
Processing uf20-91\uf20-0576.cnf
Processing uf20-91\uf20-0577.cnf
Processing uf20-91\uf20-0578.cnf
Processing uf20-91\uf20-0579.cnf
Processing uf20-91\uf20-058.cnf
Processing uf

Processing uf20-91\uf20-078.cnf
Processing uf20-91\uf20-0780.cnf
Processing uf20-91\uf20-0781.cnf
Processing uf20-91\uf20-0782.cnf
Processing uf20-91\uf20-0783.cnf
Processing uf20-91\uf20-0784.cnf
Processing uf20-91\uf20-0785.cnf
Processing uf20-91\uf20-0786.cnf
Processing uf20-91\uf20-0787.cnf
Processing uf20-91\uf20-0788.cnf
Processing uf20-91\uf20-0789.cnf
Processing uf20-91\uf20-079.cnf
Processing uf20-91\uf20-0790.cnf
Processing uf20-91\uf20-0791.cnf
Processing uf20-91\uf20-0792.cnf
Processing uf20-91\uf20-0793.cnf
Processing uf20-91\uf20-0794.cnf
Processing uf20-91\uf20-0795.cnf
Processing uf20-91\uf20-0796.cnf
Processing uf20-91\uf20-0797.cnf
Processing uf20-91\uf20-0798.cnf
Processing uf20-91\uf20-0799.cnf
Processing uf20-91\uf20-08.cnf
Processing uf20-91\uf20-080.cnf
Processing uf20-91\uf20-0800.cnf
Processing uf20-91\uf20-0801.cnf
Processing uf20-91\uf20-0802.cnf
Processing uf20-91\uf20-0803.cnf
Processing uf20-91\uf20-0804.cnf
Processing uf20-91\uf20-0805.cnf
Processing uf20